In [37]:
import os
import numpy as np
import pandas as pd
import gc

df_train=pd.read_csv('./train.csv')
df_test=pd.read_csv('./test.csv')
df_songs=pd.read_csv('./songs.csv')
df_songs_extra=pd.read_csv('./song_extra_info.csv')
df_members=pd.read_csv('./members.csv')
df_sample_submission=pd.read_csv('./sample_submission.csv')
print(df_train.shape, df_test.shape, df_songs.shape,df_songs_extra.shape, df_members.shape)



(7377418, 6) (2556790, 6) (2296320, 7) (2295971, 3) (34403, 7)


In [38]:
from pyunpack.cli import Archive
import os
import numpy as np
os.system('apt-get install p7zip')

1

In [39]:
df_test

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio
...,...,...,...,...,...,...
2556785,2556785,XmA/cIkXJ8jZUfsUc4bBfJVWMMqmylnPW0WVkg/iz0s=,wJoWzZd7AL+qX9xZWZwRTzGRYg0Lxcl5Pe+9n5hZgAQ=,discover,Online playlist more,online-playlist
2556786,2556786,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,KCJ8BlSfRQRgB7EVuzFvg52AhR8m2fT032MzN5ewbEI=,discover,NaN,online-playlist
2556787,2556787,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,ySDsKJSnhFMESzC9mBhY0hnFEEBWDYx0FOMNcYDLb/A=,discover,NaN,online-playlist
2556788,2556788,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,WXH8kL8e+0H2jY+s2Y6FpHyyHdaMYV3b6yfDa3FAt9U=,discover,NaN,online-playlist


In [40]:
#'song_id', 'song_length', 'genre_ids', 'artist_name', 'composer','lyricist', 'language'
df_songs['language']=df_songs['language'].astype('category')
df_songs=df_songs.fillna({'composer': 'NA', 'lyricist': 'NA'})

df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NA,NA,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [41]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

df_songs_extra['song_year'] = df_songs_extra['isrc'].apply(isrc_to_year)
df_songs_extra.head()

,song_id,name,isrc,song_year
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043,2012.0
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015,2016.0
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303,2008.0
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446,2013.0
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001,2013.0


In [42]:
df_members=df_members.fillna({'gender': 'NA'})
df_members.registration_init_time = pd.to_datetime(df_members.registration_init_time, format='%Y%m%d', errors='ignore')
df_members.expiration_date = pd.to_datetime(df_members.expiration_date, format='%Y%m%d', errors='ignore')
df_members['registration_year'] = df_members['registration_init_time'].dt.year
df_members['registration_month'] = df_members['registration_init_time'].dt.month

df_members['expiration_year'] = df_members['expiration_date'].dt.year
df_members['expiration_month'] = df_members['expiration_date'].dt.month

fix_age = lambda x: min(max(x, 10), 60) if 10 <= x <= 60 else np.nan
df_members['age'] =df_members['bd'].apply(fix_age)
df_members.drop('bd', axis=1, inplace=True)

df_members


,msno,city,gender,registered_via,registration_init_time,expiration_date,registration_year,registration_month,expiration_year,expiration_month,age
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,NA,7,2011-08-20,2017-09-20,2011,8,2017,9,NaN
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,NA,7,2015-06-28,2017-06-22,2015,6,2017,6,NaN
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,NA,4,2016-04-11,2017-07-12,2016,4,2017,7,NaN
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,NA,9,2015-09-06,2015-09-07,2015,9,2015,9,NaN
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,NA,4,2017-01-26,2017-06-13,2017,1,2017,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...
34398,Wwd/cudKVuLJ3txRVxlg2Zaeliu+LRUfiBmfrnxhRCY=,1,NA,7,2013-11-11,2017-09-10,2013,11,2017,9,NaN
34399,g3JGnJX6Hg50lFbrNWfsHwCUmApIkiv2M8sXOaeXoIQ=,4,male,3,2014-10-24,2017-05-18,2014,10,2017,5,18.0
34400,IMaPMJuyN+ip9Vqi+z2XuXbFAP2kbHr+EvvCNkFfj+o=,1,NA,7,2013-08-02,2017-09-08,2013,8,2017,9,NaN
34401,WAnCAJjUty9Stv8yKtV7ZC7PN+ilOy5FX3aIJgGPANM=,1,NA,7,2015-10-20,2017-09-20,2015,10,2017,9,NaN


In [43]:
#song_cols = ['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer', 'language']
df_songs = df_songs.merge(df_songs_extra , on = 'song_id' , how='left')
print(df_songs.columns)
df_songs.columns=['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'language', 'song_name', 'isrc', 'song_year']

Index(['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'language', 'name', 'isrc', 'song_year'],
      dtype='object')


In [44]:
songs_columns=['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer','lyricist', 'language', 'song_year']
df_train = df_train.merge(df_songs[songs_columns], on='song_id', how='left')
for col in df_train.columns:
    if col not in ['song_id','msno','song_length']:
        print(df_train[col].value_counts(), "\n")
        
df_train['language']=df_train['language'].astype('category')
gc.collect()

my library      3684730
discover        2179252
search           623286
radio            476701
listen with      212266
explore          167949
notification       6185
settings           2200
Name: source_system_tab, dtype: int64 

Local playlist more     3228202
Online playlist more    1294689
Radio                    474467
Album more               420156
Search                   298487
Artist more              252429
Discover Feature         244246
Discover Chart           213658
Others profile more      201795
Discover Genre            82202
My library                75980
Explore                   72342
Unknown                   54170
Discover New              15955
Search Trends             13632
Search Home               13482
My library_Search          6451
Self profile more           212
Concert                      47
Payment                      12
Name: source_screen_name, dtype: int64 

local-library             2261399
online-playlist           1967924
local-playlist     

149

In [45]:
df_train = df_train.merge(df_members , on = 'msno' , how='left')
del df_songs
del df_members

In [46]:
dict_count_song_played_train = {k: v for k, v in df_train['song_id'].value_counts().iteritems()}

def return_number_played(x):
    try:
        return dict_count_song_played_train[x]
    except KeyError:
        try:
            return dict_count_song_played_test[x]
        except KeyError:
            return 0
df_train['number_of_time_played'] = df_train['song_id'].apply(lambda x: return_number_played(x))
df_train

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,...,gender,registered_via,registration_init_time,expiration_date,registration_year,registration_month,expiration_year,expiration_month,age,number_of_time_played
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,Dan Smith| Mark Crew,...,NA,7,2012-01-02,2017-10-05,2012,1,2017,10,NaN,215
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,NA,...,female,9,2011-05-25,2017-09-11,2011,5,2017,9,24.0,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,...,female,9,2011-05-25,2017-09-11,2011,5,2017,9,24.0,4
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512.0,1019,Soundway,Kwadwo Donkoh,...,female,9,2011-05-25,2017-09-11,2011,5,2017,9,24.0,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,...,NA,7,2012-01-02,2017-10-05,2012,1,2017,10,NaN,412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,my library,Local playlist more,local-playlist,1,267958.0,1259,Dr. Dre,Andre R. Young| Melvin Charles Bradford| Shawn...,...,female,9,2004-12-18,2017-09-30,2004,12,2017,9,26.0,59
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,search,Search,song,0,227404.0,465,R. City feat. Adam Levine,Theron Thomas| Timothy Thomas| Lukasz Gottwald...,...,NA,7,2013-06-27,2017-10-06,2013,6,2017,10,NaN,1010
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,search,Search,song,1,258298.0,465,兄弟本色G.U.T.S. (姚中仁、張震嶽、頑童MJ116),姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,...,NA,7,2013-06-27,2017-10-06,2013,6,2017,10,NaN,10500
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,discover,Discover Genre,online-playlist,1,524146.0,2122,Brad Mehldau Trio,NA,...,NA,9,2015-02-13,2017-09-18,2015,2,2017,9,NaN,11


In [47]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix, classification_report
import lightgbm as lgb
import lightgbm as lgbm
import xgboost as xgb

In [48]:
label_encoder = LabelEncoder()

for i in df_train.columns :
    df_train[i] = label_encoder.fit_transform(df_train[i])

    
X = df_train.drop('target', axis = 1)
y = df_train.target
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)

del df_train

In [ ]:
gc.collect()

models={
    'LGB': lgbm.LGBMClassifier(objective='binary',n_jobs=5,max_depth=10,n_estimators=500),
    'XGB':xgb.XGBClassifier(n_jobs=5,max_depth=10, n_estimators=500)
}


for k,model in models.items():
    print(k)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    print(classification_report(y_train, train_pred))
    print(classification_report(y_val, val_pred))
    print()
    print("Accuracy :", accuracy_score(y_val, val_pred))
    print("ROC :", roc_auc_score(y_val, val_pred))
    
    print("==================================================")
    print()

LGB
              precision    recall  f1-score   support

           0       0.68      0.63      0.65   2746912
           1       0.66      0.71      0.68   2786151

    accuracy                           0.67   5533063
   macro avg       0.67      0.67      0.67   5533063
weighted avg       0.67      0.67      0.67   5533063

              precision    recall  f1-score   support

           0       0.68      0.63      0.65    915850
           1       0.66      0.71      0.68    928505

    accuracy                           0.67   1844355
   macro avg       0.67      0.67      0.67   1844355
weighted avg       0.67      0.67      0.67   1844355


Accuracy : 0.6685182624820059
ROC : 0.6682377384325995

XGB
